In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from osgeo import gdal, osr, ogr  # necessario intalar a whell manualmente https://www.lfd.uci.edu/~gohlke/pythonlibs/
import math
import time
import pyproj
#import rasterio
#from rasterio import features

In [2]:
file = 'gpd.csv'

MERCATOR = pyproj.CRS('epsg:3857')
WGS84 = pyproj.CRS('epsg:4326')
SAD1969  = pyproj.CRS('esri:102033')

df = pd.read_csv(file)

 

df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')

wkt_geom = [i.wkt for i in gdf.geometry]
#gdf.explore()

In [3]:
x_size = gdf.total_bounds[2]-gdf.total_bounds[0]
y_size = gdf.total_bounds[3]-gdf.total_bounds[1]
x_0 = gdf.total_bounds[0]
y_0 = gdf.total_bounds[3]

 

pix_size = 0.000002
pixels_in_x = math.ceil(x_size/pix_size)
pixels_in_y = math.ceil(y_size/pix_size)

 
sr_wkt = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'
sr = osr.SpatialReference( sr_wkt )


# Create a memory raster to rasterize into.

 

target_ds = gdal.GetDriverByName('MEM').Create('', pixels_in_x, pixels_in_y, 3, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_0, pix_size, 0, y_0, 0, -pix_size))

 

# Create a memory layer to rasterize from.

 
rast_ogr_ds  = ogr.GetDriverByName('Memory').CreateDataSource('wrk')
rast_mem_lyr = rast_ogr_ds.CreateLayer('poly', srs=sr)

 

# Add polygons and linestrings.
wkt_geom = [i.wkt for i in gdf[gdf['cor'] == 'produzindo_rastro'].geometry]
#wkt_geom = ['POLYGON((0 0 0,0 10 0,10 10 0,10 0 0,0 0 0))']

 

for g in wkt_geom:
    feat = ogr.Feature(rast_mem_lyr.GetLayerDefn())
    feat.SetGeometryDirectly(ogr.Geometry(wkt = g))
    rast_mem_lyr.CreateFeature(feat)

 
data = gdf.value.tolist()

gdal.RasterizeLayer(target_ds, [1,2,3], rast_mem_lyr)
#gdal.GetDriverByName('PNG').CreateCopy('rasterize.png',target_ds)

 

proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'
target_ds.SetProjection(proj)

 

tiffdriver = gdal.GetDriverByName('GTiff')
tiffdriver.CreateCopy('rasterize.tif', target_ds)

 

pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize.png', target_ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001F2081C2880> >

In [10]:
test = gdal.Open('rasterize.tif')
test1 = gdal.Open('sp27.tif')

In [11]:
test.GetProjection()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [12]:
test1.GetProjection()

'PROJCS["NAD27 / California zone VI",GEOGCS["NAD27",DATUM["North_American_Datum_1927",SPHEROID["Clarke 1866",6378206.4,294.978698213898,AUTHORITY["EPSG","7008"]],AUTHORITY["EPSG","6267"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4267"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",32.1666666666667],PARAMETER["central_meridian",-116.25],PARAMETER["standard_parallel_1",33.8833333333333],PARAMETER["standard_parallel_2",32.7833333333333],PARAMETER["false_easting",2000000],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26746"]]'

In [9]:
import platform
print(platform.system(),)
print(platform.machine(),)
print(platform.platform(),)
print(platform.uname(),)
print(platform.version(),)
print(platform.mac_ver(),)

Windows
AMD64
Windows-10-10.0.19045-SP0
uname_result(system='Windows', node='C118', release='10', version='10.0.19045', machine='AMD64')
10.0.19045
('', ('', '', ''), '')
